In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from ipyannotations.images import PointAnnotator

In [ ]:
images = []

dataset_path = "/PATH/TO/DATA"
for category in [c for c in os.listdir(dataset_path) if c != "empty"]:
    files = [file for file in os.listdir(
        os.path.join(dataset_path, category, "cropped_rgb")) if ".png" in file
    ]
    groups = {}
    for file in files:
        *prefix, _ = os.path.splitext(file)[0].split("_")
        prefix_str = "_".join(prefix)
        if prefix_str not in groups:
            groups[prefix_str] = [file]
        else:
            groups[prefix_str].append(file)

    for prefix, files in sorted(groups.items()):
        images.append(os.path.join(dataset_path, category, "cropped_rgb", sorted(files)[0]))

In [ ]:
import cv2

def visualize_gt(img, sample):
    gt_colors = [(255, 0, 0), (0, 255, 0)]
    for arm, gt_color in zip(
        ["left", "right"], gt_colors
    ):
        if f"{arm}_pick" in sample:
            pick = (
                sample[f"{arm}_pick"]
            )
            place = (
                sample[f"{arm}_place"]
            )
            img = _pick_place_viz(
                img,
                pick,
                place,
                color=gt_color,
            )
    return img

def _pick_place_viz(img, picks, places, color):
    if not isinstance(picks, list) and len(picks.shape) == 1:
        picks = [picks]
        places = [places]
    for pick, place in zip(picks, places):
        if pick[0] >= 0:
            cv2.circle(
                img=img,
                center=(round(pick[0]), round(pick[1])),
                radius=3,
                color=color,
                thickness=2,
            )
        if place[0] >= 0:
            cv2.arrowedLine(
                img=img,
                pt1=(round(pick[0]), round(pick[1])),
                pt2=(round(place[0]), round(place[1])),
                color=color,
                thickness=2,
            )
    return img

# Annotation instructions

Bear in mind that the origin of the manipulation for both arms has to be in the cloth region.
The next image is the state to which you try to reach doing your manipulation. If the next image is unrelated there is no need to label anything.

The instructions are:
1. With "left-arm" class selected (done by default), click first the "from" point and then the "to" point for the left arm.
2. Select "right-arm" class
3. Click first the "from" point and then the "to" point for the right arm.
4. If you want to annotate another bimanual manipulation, select "left-arm" class and go to 1.

In [ ]:
idx = -1 #Add index of image to annotate

image = images[idx]

widget = PointAnnotator(options=["left-arm", "right-arm"])
widget.display(image)
widget

In [ ]:
if widget.data:
    assert len(widget.data) % 4 == 0, f"Error for widget {i} with length {len(widget.data)}"
    num_chunks = len(widget.data) // 4
    
    coords = []
    for chunk_idx in range(num_chunks):
        chunk = widget.data[chunk_idx * 4:(chunk_idx + 1) * 4]
        assert "left" in chunk[0]["label"] and "left" in chunk[1]["label"] and "right" in chunk[2]["label"] and "right" in chunk[3]["label"], f"Failed for widget{i}"
        from_left = chunk[0]["coordinates"]
        from_right = chunk[2]["coordinates"]

        *path, _, filename = image.split(os.path.sep)
        annotations_dir = os.path.join(*path, "cropped_annotations")
        os.makedirs(annotations_dir, exist_ok=True)

        mask = (np.asarray(Image.open(os.path.join(*path, "cropped_mask", filename)))[:, :, 0] / 255).astype(bool)
        if mask[from_left[1], from_left[0]] and mask[from_right[1], from_right[0]]:
            *prefix, _ = os.path.splitext(filename)[0].split("_")
            annotations_file = os.path.join(annotations_dir, "_".join(prefix) + ".npy")
            
            coords.append(np.array([*[point["coordinates"] for point in chunk]]).flatten())
    if coords:
        coords = np.array(coords)
        if os.path.isfile(annotations_file):
            print(f"Updating {annotations_file}")
            saved_coords = np.load(annotations_file)
            if len(saved_coords.shape) == 1:
                saved_coords = saved_coords[None, :]
            np.save(annotations_file, np.unique(np.r_[saved_coords, coords]))
        else:
            print(f"Creating {annotations_file}")
            np.save(annotations_file, coords)
    else:
        print(f"Erronous data")
else:
    print(f"\tWidget has no data")

In [ ]:
for category in [c for c in os.listdir(dataset_path) if c != "empty"]:
    if os.path.isdir(os.path.join(dataset_path, category, "cropped_annotations")):
        for file in os.listdir(
            os.path.join(dataset_path, category, "cropped_annotations")
        ):
            if os.path.isfile(os.path.join(dataset_path, category, "cropped_annotations", file)):
                annotation = np.load(os.path.join(dataset_path, category, "cropped_annotations", file))
                if len(annotation.shape) == 1:
                    annotation = annotation[None, :]
        
                img = np.asarray(Image.open(os.path.join(dataset_path, category, "cropped_rgb", file.replace(".npy", "_0000.png"))))
                
                visualization = visualize_gt(img, {
                    "left_pick": annotation[:, [0, 1]],
                    "left_place": annotation[:, [2, 3]],
                    "right_pick": annotation[:, [4, 5]],
                    "right_place": annotation[:, [6, 7]],
                })
        
                os.makedirs(os.path.join(dataset_path, category, "cropped_viz"), exist_ok=True)
                Image.fromarray(visualization).save(os.path.join(dataset_path, category, "cropped_viz", file.replace(".npy", ".png")))